Import Libraries

In [1]:
!pip install tweepy --upgrade
!pip install contractions
!pip install nltk
!pip install stop_words
!pip install emoji

import tweepy

import requests
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.insert(0,'/content/gdrive/My Drive/Colab Notebooks')
import config

import contractions
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words

import emoji
import re
from emoji import demojize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 3.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 12.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [2]:
client = tweepy.Client(config.bearer_token) #Insert twitter bearer token

In [3]:
# Clean data function
def clean_tweets(tweets, depressed):

  # remove certain words only in tweets w/the word depression 
  if depressed == True:
    # Include tweets only containing the following regular expressions
    tweets = tweets.loc[tweets.tweet.str.contains("I\s|I'm|\sme\s|feel|felt|i'm|\smy\s"),:] 
  
  # Remove certain words in random tweets
  elif depressed == False:
    #Remove tweets indicating depression or anxiety
    tweets = tweets.loc[tweets.tweet.str.contains("""@|depress|anxiety""") == False,:]


  # Lowercase
  tweets['tweet'] = tweets['tweet'].str.lower()

  # Remove Contractions
  revised_tweet_list = []
  for tweet in tweets.tweet: 
    # create an empty list of expanded words
    expanded_words = []   
    for word in tweet.split():
      #  contractions.fix expands the words
      expanded_words.append(contractions.fix(word))  
    expanded_tweet = ' '.join(expanded_words)
    revised_tweet_list.append(expanded_tweet)
  tweets['expanded_tweet'] = revised_tweet_list
  return(tweets)

##Get depressing tweets

In [4]:
# Select tweets with words depressed, antidepressants, and anxiety; exclude retweets, replies, and media
query_list = ["depressed lang:en -is:retweet -has:media -is:reply", "depression lang:en -is:retweet -has:media -is:reply", "antidepressants lang:en -is:retweet -has:media -is:reply", "antidepressant lang:en -is:retweet -has:media -is:reply", "anxiety lang:en -is:retweet -has:media -is:reply"]
tweets_list = []
id_list = []

for query in query_list:
  tweets = tweepy.Paginator(client.search_recent_tweets, query=query,
                              tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=1200)
  for tweet in tweets:
      tweets_list.append(tweet.text)
      id_list.append(tweet.id)

df = pd.DataFrame(tweets_list, columns=['tweet'])
df['id'] = id_list
pd.options.display.max_colwidth = 300  # set a value as your need

*Here, I omit tweets that use the keywords (depression, anxiety, etc) casually or refer to an unofficial diagnosis of depression*

* "i miss mexico so much it is making me depressed 😩	"
* "i sent a california meme to my s/o and his dad because we have a group chat and instead of laughing because it is true, i am pretty sure i just made his dad more depressed about the state of california"
* "i could not be happier with my mothers improvement. it took me to tell her to stop being so unhappy and depressed! things are better now, she is in a much better place. amen"
* "i have been hair depressed but i get my hair done tomorrow so i am good"
* "i still cannot believe my first research proposal was approved! we will study brain development in depressed adolescents!"
* "i had post concert depression for a couple months since trinity of terror. last night was just what i needed!! hollywood undead and papa roach was so good. at one point it felt like my vocal chords fell out of my mouth."*

In [6]:
# Manually remove irrelevant tweets
df3 = df.loc[df.tweet.str.contains('nor am i depressed|tickets|he got depress|episode|does not give me the anxiety|cannot stop listening to|here are.*tips|album|totsuki|hopes it helps them understand|turns out i just needed|him into.*depression|ferrari|jimi hendrix|hair depress|hate a depressed|them snapchat memories|turns out i just needed|everyone seems depressed|was a depressed kid|he will inevitably get depressed|concert|kumbe|weyes blood|deftones|depressed people|shalin|veronica|depressed person|boobs are my|press a video|math anxiety|notifications|dogging the gym|who quoted this|posting or sharing|boob mri|anxiety eye color|hugs can have several benefits|less and less anxious|to bali|bocchi|anxiety money|never afraid of flying|seriously depressed me|the depressed one|thorfinn|salted caramel|taylor swift exists|research tells us|cough syrup as|yonc|hook up|@|cock|be depress|are depress|depression gone|depression nap|leaving my body|priyanka|eradicate my depression than|saved me from depression|shout out to|cure for depression|depression tweets|hot.*sexy|that was before the depression|great depression|post.* depression|my depression is cured|one with depression|cure[s]? my depression|depressed him|depressed her|is.*depress|you.*depress|my.*depressed|me depressed|not depressed|false alarm|my depressed|is depressed|be depress|depressed\?|follow|lakers|s[0-9]|watch|news|tv|show|having a sick dog|gender dysphoria|stressed and depressed|stole my antidepressants|going off antidepressants|".*"|basil|pharmaceutical compan|click here|works as an antidepressant|antidepressant lamp|clinical trial|antidepressant study|he best antidepressant i have|doki doki|okhty|bruce depression|ride out a depression|hair depression|will remain my antidepressant|teenage aggression|ball of anxiety') == False,:]
df3 = df3.loc[df3.tweet.str.contains('season|not depressed| ".*"|proud|are depressed|research|you depressed|spotify|breast|4k|hair depress|nets fan|amen|drunk and depress|@|state of cali|fake depress|thewalkingdead|snap flashbacks|hate waiting for zoom|staying in this house|depressed mode|concert depress|i could not have written it better|san remo|your period|long in seoul|disco ball|cure my depression|studies|a study|luciana|pre-sales|ocpd|security cameras|my stories|separation anxiety|muscle spasm|subtronics|shawty|namjoon|acnh|music is my medicine| classmate to depress|reach out and|match history|hanbin|unu|i miss mexico|pulling up in the park|i want a relationship') == False,:]
len(df3)

3391

In [7]:
# Get cleaned depressed tweets
df3 = clean_tweets(df, True)
df3.head()

<ipython-input-3-afc8f015d4dd>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets['tweet'] = tweets['tweet'].str.lower()
<ipython-input-3-afc8f015d4dd>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets['expanded_tweet'] = revised_tweet_list


,tweet,id,expanded_tweet
0,"i’ve been feeling depressed from this morning and don’t really have an appetite... but if it’s something with chocolate in it, i think i might be able to eat it because it’s my favorite.",1624842428162375685,"i have been feeling depressed from this morning and do not really have an appetite... but if it is something with chocolate in it, i think i might be able to eat it because it is my favorite."
1,your mess in my chest has me stressed and depressed,1624842421388574720,your mess in my chest has me stressed and depressed
3,“i already eaten 56 freaking happy meals\n\nand i am still depressed.”,1624842361552486400,“i already eaten 56 freaking happy meals and i am still depressed.”
5,tomorrow is sunflower week and i have been to depressed to do anything for it…maybe i can try to continue the two i have set up tonight…,1624842240089550850,tomorrow is sunflower week and i have been to depressed to do anything for it…maybe i can try to continue the two i have set up tonight…
7,"very depressed today. another futile battle with wife.. relationship sinking into the pits. no meeting of minds on covid caution.\nshe is right (in her terms) - lots of ppl we know had 'mild' covid, and of them i'm the closest to a visible case of lc.. so it's just me! 🤦‍♂️\n1/2",1624842172414451714,"very depressed today. another futile battle with wife.. relationship sinking into the pits. no meeting of minds on covid caution. she is right (in her terms) - lots of people we know had 'mild' covid, and of them i am the closest to a visible case of lc.. so it is just me! 🤦‍♂️ 1/2"


In [8]:
# Convert any emoji to unicode
df3['tweet'] = df3.apply(lambda row: (row['expanded_tweet']).encode('unicode-escape').decode('ASCII'), axis=1)

# Subset to DF where tweets contain unicode
df3 = df3[df3['tweet'].str.contains("\\\\U")]
df3.head()

# Remove all text and keep unicode
df3['unicoded'] = df3['tweet'].str.extract(r"\(\\\\U([A-Za-z0-9 _]+)\)")
df3['unicoded'] = df3['tweet'].str.replace(r"\(\\\\U([A-Za-z0-9 _]+)\)", "")

<ipython-input-8-9dd512ea7ccb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['tweet'] = df3.apply(lambda row: (row['expanded_tweet']).encode('unicode-escape').decode('ASCII'), axis=1)
<ipython-input-8-9dd512ea7ccb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['unicoded'] = df3['tweet'].str.extract(r"\(\\\\U([A-Za-z0-9 _]+)\)")
<ipython-input-8-9dd512ea7ccb>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df3['unicoded'] = df3['tweet'].str.re

In [9]:
# Manually remove irrelevant tweets
#df3 = df3.loc[df3.unicoded.str.contains('season|not depressed| ".*"|proud|are depressed|research|you depressed|spotify|breast|4k|hair depress|nets fan|amen|drunk and depress|@|state of cali|fake depress|thewalkingdead|snap flashbacks|hate waiting for zoom|staying in this house|depressed mode|concert depress|i could not have written it better|san remo|your period|long in seoul|disco ball|cure my depression|studies|a study|luciana|pre-sales|ocpd|security cameras|my stories|separation anxiety|muscle spasm|subtronics|shawty|namjoon|acnh|music is my medicine| classmate to depress|reach out and|match history|hanbin|unu|i miss mexico|pulling up in the park|i want a relationship') == False,:]
#len(df3)

In [10]:
# Only keep unicode
df3['unicode'] = df3['unicoded'].str.findall("(\\\\U[0-9a-z]{8})")
df3.head()

<ipython-input-10-7e081ed07ad9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['unicode'] = df3['unicoded'].str.findall("(\\\\U[0-9a-z]{8})")


,tweet,id,expanded_tweet,unicoded,unicode
7,"very depressed today. another futile battle with wife.. relationship sinking into the pits. no meeting of minds on covid caution. she is right (in her terms) - lots of people we know had 'mild' covid, and of them i am the closest to a visible case of lc.. so it is just me! \U0001f926\u200d\u2642...",1624842172414451714,"very depressed today. another futile battle with wife.. relationship sinking into the pits. no meeting of minds on covid caution. she is right (in her terms) - lots of people we know had 'mild' covid, and of them i am the closest to a visible case of lc.. so it is just me! 🤦‍♂️ 1/2","very depressed today. another futile battle with wife.. relationship sinking into the pits. no meeting of minds on covid caution. she is right (in her terms) - lots of people we know had 'mild' covid, and of them i am the closest to a visible case of lc.. so it is just me! \U0001f926\u200d\u2642...",[\U0001f926]
13,"you told everybody i was acting weird, but i was depressed \U0001f494 and you just was not a good friend \U0001f4af",1624841866112802816,"you told everybody i was acting weird, but i was depressed 💔 and you just was not a good friend 💯","you told everybody i was acting weird, but i was depressed \U0001f494 and you just was not a good friend \U0001f4af","[\U0001f494, \U0001f4af]"
20,man i am so depressed \U0001f62d i called off work :(,1624841360745566209,man i am so depressed 😭 i called off work :(,man i am so depressed \U0001f62d i called off work :(,[\U0001f62d]
39,"watching xtro for the first time - kind of notorious movie but i do not know much what to expect other than unhappiness. ebert said, ""xtro is an ugly, mean-spirited and despairing thriller that left me thoroughly depressed."" \U0001f606\U0001f44d",1624840530961084416,"watching xtro for the first time - kind of notorious movie but i do not know much what to expect other than unhappiness. ebert said, ""xtro is an ugly, mean-spirited and despairing thriller that left me thoroughly depressed."" 😆👍","watching xtro for the first time - kind of notorious movie but i do not know much what to expect other than unhappiness. ebert said, ""xtro is an ugly, mean-spirited and despairing thriller that left me thoroughly depressed."" \U0001f606\U0001f44d","[\U0001f606, \U0001f44d]"
43,i just watched a tiktok where this woman was talking about how she moved to a new country and thought she was \u201cdepressed\u201d. the depression is now 8 years old and sassy as can be lmao \U0001f62d,1624840380343541760,i just watched a tiktok where this woman was talking about how she moved to a new country and thought she was “depressed”. the depression is now 8 years old and sassy as can be lmao 😭,i just watched a tiktok where this woman was talking about how she moved to a new country and thought she was \u201cdepressed\u201d. the depression is now 8 years old and sassy as can be lmao \U0001f62d,[\U0001f62d]


### Get random tweets w/emoji

In [11]:
# Get tweets
random_tweets_list = []
random_id_list = []

random_tweets = tweepy.Paginator(client.search_recent_tweets, query = "lang:en -is:retweet -has:media -the the",
                              tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=8000)
for random_tweet in random_tweets:
    random_tweets_list.append(random_tweet.text)
    random_id_list.append(random_tweet.id)

rdf = pd.DataFrame(random_tweets_list, columns=['tweet'])
rdf['id'] = random_id_list
pd.options.display.max_colwidth = 300

rdf2 = clean_tweets(rdf, False)

rdf2.head()

<ipython-input-3-afc8f015d4dd>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets['tweet'] = tweets['tweet'].str.lower()
<ipython-input-3-afc8f015d4dd>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets['expanded_tweet'] = revised_tweet_list


,tweet,id,expanded_tweet
1,*backflip* #anibot,1624842702255947776,*backflip* #anibot
3,your latest follower is lowkey interested in uu — who? dm me https://t.co/wl764l2sv4,1624842702251560960,your latest follower is lowkey interested in uu — who? dm me https://t.co/wl764l2sv4
5,i like my ass to be between the color of baked chicken or well done grilled steak .,1624842702239174656,i like my ass to be between the color of baked chicken or well done grilled steak .
7,#bbtitans ebuka asked all the right questions we don't like peace 🤣,1624842702213767170,#bbtitans ebuka asked all the right questions we do not like peace 🤣
9,"$xrp guys, don't miss the next move in a few hours... https://t.co/mrbiy7k2qu.",1624842702205632513,"$xrp guys, do not miss the next move in a few hours... https://t.co/mrbiy7k2qu."


In [12]:
# Convert emoji to unicode
rdf2['expanded_tweet'] = rdf2.apply(lambda row: (row['expanded_tweet']).encode('unicode-escape').decode('ASCII'), axis=1)

# Subset to DF where tweets contain unicode
rdf2 = rdf2[rdf2['expanded_tweet'].str.contains("\\\\U")]
rdf2.head()

# Remove all text and keep unicode
rdf2['unicoded'] = rdf2['expanded_tweet'].str.extract(r"\(\\\\U([A-Za-z0-9 _]+)\)")
rdf2['unicoded'] = rdf2['expanded_tweet'].str.replace(r"\(\\\\U([A-Za-z0-9 _]+)\)", "")

<ipython-input-12-692ea94b270d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdf2['expanded_tweet'] = rdf2.apply(lambda row: (row['expanded_tweet']).encode('unicode-escape').decode('ASCII'), axis=1)
<ipython-input-12-692ea94b270d>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  rdf2['unicoded'] = rdf2['expanded_tweet'].str.replace(r"\(\\\\U([A-Za-z0-9 _]+)\)", "")


In [13]:
len(rdf2)

553

In [14]:
# Only keep unicode
rdf2['unicode'] = rdf2['unicoded'].str.findall("(\\\\U[0-9a-z]{8})")
rdf2.head()

,tweet,id,expanded_tweet,unicoded,unicode
7,#bbtitans ebuka asked all the right questions we don't like peace 🤣,1624842702213767170,#bbtitans ebuka asked all the right questions we do not like peace \U0001f923,#bbtitans ebuka asked all the right questions we do not like peace \U0001f923,[\U0001f923]
45,lmfaooo! they were confident they weren’t leaving. 😂 https://t.co/5rbguohuky,1624842701966389249,lmfaooo! they were confident they were not leaving. \U0001f602 https://t.co/5rbguohuky,lmfaooo! they were confident they were not leaving. \U0001f602 https://t.co/5rbguohuky,[\U0001f602]
61,go join😈 https://t.co/jwkstcgckk,1624842701878304768,go join\U0001f608 https://t.co/jwkstcgckk,go join\U0001f608 https://t.co/jwkstcgckk,[\U0001f608]
76,i love cardi. she inspires me to be stripper and mom and wife and entertainer and overcome being and getting out proverty and not being worried about my sex worker past. 🥹 https://t.co/lkgur2tjww,1624842701811200002,i love cardi. she inspires me to be stripper and mom and wife and entertainer and overcome being and getting out proverty and not being worried about my sex worker past. \U0001f979 https://t.co/lkgur2tjww,i love cardi. she inspires me to be stripper and mom and wife and entertainer and overcome being and getting out proverty and not being worried about my sex worker past. \U0001f979 https://t.co/lkgur2tjww,[\U0001f979]
102,man idk is this actually smart by astralis💀 double poison maybe they just run them down,1624842701622460420,man i do not know is this actually smart by astralis\U0001f480 double poison maybe they just run them down,man i do not know is this actually smart by astralis\U0001f480 double poison maybe they just run them down,[\U0001f480]


In [15]:
# Add target variables to tweets
df3['depressed'] = 1
rdf2['depressed'] = 0

# Balance data
rdf2 = rdf2[0:len(df3)]

# Append the datasets together
emojis_df = df3[['tweet', 'id', 'unicode', 'depressed']].append(rdf2[['tweet', 'id', 'unicode', 'depressed']])

# Create a df of all emojis in all tweets
emoji_list = []
ids = []
depressed_list = []
for index,item in emojis_df.iterrows():
  for emojis in item.unicode:
    emoji_list.append(emojis)
    ids.append(item.id)
    depressed_list.append(item.depressed)

emojis_df = pd.DataFrame(list(zip(emoji_list, ids, depressed_list)), columns =['unicode', 'id', 'depressed'])

# Convert unicode to emoji
emojis_df['emoji'] = emojis_df.apply(lambda row: (row['unicode']).encode('ascii').decode('unicode-escape').encode('utf-16', 'surrogatepass').decode('utf-16'), axis=1)

# Get frequencies of emojis in tweets by classification
emojis_df.groupby(['depressed']).emoji.value_counts()

depressed  emoji
0          🔹        240
           😭        139
           😂        102
           🤣         48
           🟩         34
                   ... 
1          🧐          1
           🧚          1
           🪴          1
           🫥          1
           🫰          1
Name: emoji, Length: 459, dtype: int64

In [16]:
demoji_list = []
# Convert emoji to a text format
for emoji in emojis_df.emoji:
  demoji = demojize(emoji)
  demoji = re.sub(r":", r"", demoji)
  demoji_list.append(demoji)

emojis_df['demoji'] = demoji_list

In [17]:
# Export data to CSV
emojis_df.to_csv('emojis_df.csv')
emojis_df.head()

,unicode,id,depressed,emoji,demoji
0,\U0001f926,1624842172414451714,1,🤦,person_facepalming
1,\U0001f494,1624841866112802816,1,💔,broken_heart
2,\U0001f4af,1624841866112802816,1,💯,hundred_points
3,\U0001f62d,1624841360745566209,1,😭,loudly_crying_face
4,\U0001f606,1624840530961084416,1,😆,grinning_squinting_face
